In [9]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import operator




def wrap_ttest(df, label_column, comparison_columns=None, alpha=.05, return_all=False, correction_method='bonferroni', mincount=3, pval_return_corrected=True):
    try:
        '''Verify precondition that label column exists and has exactly 2 unique values'''
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
        
        '''Partition dataframe into two sets, one for each of the two unique values from the label column'''
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]

        '''If no comparison columns specified, use all columns except the specified labed column'''
        if not comparison_columns:
            comparison_columns = list(df.columns)
            comparison_columns.remove(label_column)

        '''Determine the number of real valued columns on which we will do t-tests'''
        number_of_comparisons = len(comparison_columns)

        '''Store comparisons and p-values in two arrays'''
        comparisons = []
        pvals = []
        
        '''Loop through each comparison column, perform the t-test, and record the p-val'''
        
        for column in comparison_columns:
            if len(partition1[column].dropna(axis=0)) <= mincount:
                continue
            elif len(partition2[column].dropna(axis=0)) <= mincount:
                continue
            else:
                stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
                comparisons.append(column)
                pvals.append(pval)
                
        '''Correct for multiple testing to determine if each comparison meets the new cutoff'''
        results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
        reject = results[0]

        '''Format results in a pandas dataframe'''
        results_df = pd.DataFrame(columns=['Comparison','P_Value'])

        '''If return all, add all comparisons and p-values to dataframe'''
        if return_all:
            if pval_return_corrected:
                results_df['Comparison'] = comparisons
                results_df['P_Value'] = results[1]

            else:
                results_df['Comparison'] = comparisons
                results_df['P_Value'] = pvals

            '''Else only add significant comparisons'''
        else:
            for i in range(0, len(reject)):
                if reject[i]:
                    if pval_return_corrected:
                        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':results[1][i]}, ignore_index=True)
                    else:
                        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


        '''Sort dataframe by ascending p-value'''
        results_df = results_df.sort_values(by='P_Value', ascending=True)
        results_df = results_df.reset_index(drop=True)

        '''If results df is not empty, return it, else return None'''
        if len(results_df) > 0:
            return results_df
        else:
            print("Empty")
            return results[1]


    except:
        print("Incorrectly Formatted Dataframe!")
        return results[1]

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

In [11]:
en = cptac.Endometrial()
br = cptac.Brca()
cl = cptac.Colon()

In [12]:
gene = 'PIK3CA'

# Endometrial

In [14]:
mut = en.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E545K', 'H1047R'])
phos = en.get_phosphoproteomics(tissue_type="tumor")
phos  = cptac.utils.reduce_multiindex(phos, flatten=True)
phos

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


Name,AAAS_S495,AAAS_S541,AAAS_Y485,AACS_S618,AAED1_S12,AAGAB_S310,AAGAB_S311,AAK1_S14,AAK1_S18,AAK1_S20,...,ZZZ3_S397,ZZZ3_S411,ZZZ3_S420,ZZZ3_S424,ZZZ3_S426,ZZZ3_S468,ZZZ3_S89,ZZZ3_T415,ZZZ3_T418,ZZZ3_Y399
C3L-00006,NaN,NaN,NaN,-0.881,-1.810,NaN,NaN,NaN,-0.2420,-0.2420,...,0.18400,NaN,NaN,NaN,-0.20500,NaN,NaN,NaN,NaN,NaN
C3L-00008,NaN,NaN,NaN,NaN,0.084,NaN,NaN,-1.1100,-0.3830,-1.0900,...,-0.17100,NaN,NaN,-0.393,-0.17100,NaN,0.29,NaN,0.16050,-0.06350
C3L-00032,-0.202,NaN,NaN,NaN,-1.880,NaN,NaN,NaN,0.3820,-0.0416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C3L-00090,-0.002,NaN,-0.4070,NaN,NaN,NaN,NaN,NaN,NaN,-0.5550,...,0.13970,NaN,NaN,NaN,-0.55900,NaN,NaN,NaN,NaN,0.29800
C3L-00098,0.556,-0.0461,NaN,NaN,0.941,NaN,0.429,0.3620,0.6970,-0.0529,...,-0.15875,NaN,NaN,0.196,0.06175,NaN,NaN,NaN,NaN,-0.29000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-01520,0.398,NaN,-0.0901,NaN,NaN,NaN,NaN,NaN,NaN,-0.0845,...,-0.14750,NaN,NaN,NaN,-0.07520,NaN,NaN,NaN,NaN,-0.13800
C3N-01521,0.750,0.7040,NaN,NaN,0.209,NaN,-0.229,0.2215,-0.1200,-0.5100,...,0.33600,NaN,NaN,0.442,0.42200,NaN,NaN,NaN,NaN,0.27000
C3N-01537,0.526,NaN,NaN,NaN,-0.967,NaN,NaN,0.4700,-0.3570,-0.3140,...,-0.05860,NaN,-0.559,NaN,0.30900,NaN,NaN,NaN,NaN,NaN
C3N-01802,NaN,NaN,NaN,NaN,0.180,NaN,NaN,-0.2250,0.7010,0.1400,...,-0.13200,NaN,NaN,-0.920,-0.13200,NaN,0.00,NaN,-0.04685,0.20165


In [21]:
joined = mut.join(phos)
joined['Mutation'].unique()

#select samples containing hotspot mutations
en_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

endo_hotspot = en_hotspot_df.append(wildtype)
endo_hotspot = endo_hotspot.drop(columns = ["Mutation_Status", "Location"])


# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = endo_hotspot['Mutation'].isin(compare)
missense_wt = endo_hotspot[get]
missense_wt['Mutation'].value_counts()


# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


In [18]:
missense_wt

Name,Mutation,AAAS_S495,AAAS_S541,AAAS_Y485,AACS_S618,AAED1_S12,AAGAB_S310,AAGAB_S311,AAK1_S14,AAK1_S18,...,ZZZ3_S397,ZZZ3_S411,ZZZ3_S420,ZZZ3_S424,ZZZ3_S426,ZZZ3_S468,ZZZ3_S89,ZZZ3_T415,ZZZ3_T418,ZZZ3_Y399
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,NaN,NaN,NaN,-0.8810,-1.8100,NaN,NaN,NaN,-0.24200,...,0.1840,NaN,NaN,NaN,-0.20500,NaN,NaN,NaN,NaN,NaN
C3L-00032,Missense_Mutation,-0.20200,NaN,NaN,NaN,-1.8800,NaN,NaN,NaN,0.38200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C3L-00362,Missense_Mutation,NaN,NaN,NaN,0.8440,0.2790,NaN,NaN,NaN,-0.35500,...,0.0805,NaN,NaN,NaN,-0.10900,NaN,NaN,NaN,NaN,NaN
C3L-00601,Missense_Mutation,0.59400,NaN,NaN,0.0000,-0.9820,-0.276,NaN,NaN,-0.10900,...,0.1340,NaN,NaN,0.3200,0.32000,NaN,NaN,NaN,NaN,NaN
C3L-00605,Missense_Mutation,-0.16200,NaN,0.140,NaN,NaN,NaN,NaN,NaN,NaN,...,0.1890,NaN,NaN,NaN,0.38600,NaN,NaN,NaN,NaN,0.59600
C3L-00921,Missense_Mutation,-0.52500,NaN,NaN,0.8940,-1.4000,NaN,-0.2550,0.271,0.76100,...,-0.1440,NaN,NaN,NaN,-0.22000,NaN,NaN,0.3360,NaN,-0.13100
C3L-00947,Missense_Mutation,0.46000,NaN,NaN,NaN,-0.2650,NaN,NaN,NaN,0.60800,...,0.4830,NaN,NaN,NaN,0.48300,NaN,NaN,NaN,NaN,NaN
C3N-00323,Missense_Mutation,0.53200,NaN,NaN,NaN,-0.0342,NaN,NaN,NaN,-0.44700,...,0.6280,NaN,0.20000,-0.8060,-0.80600,NaN,NaN,NaN,0.635000,0.20000
C3N-00324,Missense_Mutation,-0.79100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.28100,...,-0.3435,NaN,NaN,NaN,-0.09940,NaN,NaN,NaN,NaN,NaN


In [22]:
wrap_ttest(missense_wt, "Mutation", return_all=True, mincount=7)

,Comparison,P_Value
0,CAMK2D_S506,0.485209
1,AAAS_S495,1.000000
2,RAB12_S106,1.000000
3,RAB11FIP5_T287,1.000000
4,RAB11FIP5_S538,1.000000
...,...,...
22111,HDGFL2_S676,1.000000
22112,HDGFL2_S649,1.000000
22113,HDGFL2_S637,1.000000
22114,HDGFL3_S122,1.000000


# Brca

In [6]:
phosphoproteomics = br.get_phosphoproteomics(tissue_type="tumor")
fil = phosphoproteomics.columns.get_level_values('Name').duplicated(False)
duplicates = phosphoproteomics[phosphoproteomics.columns[fil]]
no_duplicates = phosphoproteomics[phosphoproteomics.columns[~fil]]
no_duplicates
duplicates

Name                                            ACIN1  \
Site                                             T682   
Peptide              DSSTSYTETKDPSSGQEVAtPPVPQLQVCEPK   
Database_ID NP_001158286.1|NP_001158287.1|NP_055792.1   
CPT000814                                         NaN   
CPT001846                                         NaN   
X01BR001                                       0.5208   
X01BR008                                       0.7745   
X01BR009                                       0.3286   
...                                               ...   
X21BR001                                       0.5478   
X21BR002                                       0.6328   
X21BR010                                      -0.3992   
X22BR005                                       0.2388   
X22BR006                                          NaN   

Name                                                   \
Site                                                    
Peptide                                                 
Database_ID NP_055792.1|NP_001158286.1|NP_001158287.1   
CPT000814                                     -0.5246   
CPT001846                                     -0.2830   
X01BR001                                      -0.9150   
X01BR008                                      -0.2049   
X01BR009                                       0.9288   
...                                               ...   
X21BR001                                      -0.8384   
X21BR002                                      -0.5523   
X21BR010                                      -0.5157   
X22BR005                                      -0.2604   
X22BR006                                       0.1160   

Name                                                        APBB2  \
Site                                                         S229   
Peptide                                   PNRPQSSPEDGQVATVSSsPETK   
Database_ID NP_001159522.1|NP_001317585.1|NP_775098.2|NP_004298.1   
CPT000814                                              -3.6146      
CPT001846                                              -1.4654      
X01BR001                                               -0.1102      
X01BR008                                                   NaN      
X01BR009                                                   NaN      
...                                                        ...      
X21BR001                                                   NaN      
X21BR002                                                   NaN      
X21BR010                                                1.5729      
X22BR005                                               -0.6911      
X22BR006                                                   NaN      

Name                                                               \
Site                                                                
Peptide                                                             
Database_ID NP_004298.1|NP_001159522.1|NP_001317585.1|NP_775098.2   
CPT000814                                              -2.8645      
CPT001846                                              -1.2482      
X01BR001                                               -1.0674      
X01BR008                                               -0.8557      
X01BR009                                               -0.6842      
...                                                        ...      
X21BR001                                                1.4928      
X21BR002                                                1.2035      
X21BR010                                                1.1881      
X22BR005                                               -0.1988      
X22BR006                                                0.5343      

Name                                                                                        ATXN2L  \
Site                                                                                          S496   
Peptide     

In [10]:
#DEALING WITH DUPLICATES
#get the pval and min for duplicates

duplicates = br.reduce_multiindex(duplicates, flatten=True)

# We need to figure which of databaseID we want to use, for each protein


#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest

mut_type = br.get_genotype_all_vars(gene)
joined = mut_type.join(duplicates)
joined

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

brace_hotspot = br_hotspot_df.append(wildtype)
brace_hotspot = brace_hotspot.drop(columns = ["Mutation_Status", "Location"])
brace_hotspot

# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = brace_hotspot['Mutation'].isin(compare)
missense_wt = brace_hotspot[get]
missense_wt['Mutation'].value_counts()



#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
dup_df = median_diff.join(t_test)


duplicate_gene_names = list(set(duplicates.columns.get_level_values('Name'))) #get a list of genes names that are duplicated
# duplicate_gene_names

for name in duplicate_gene_names:
    gene_duplicate = dup_df[dup_df.index.str.contains(name)]
    min_row = gene_duplicate[gene_duplicate.P_Value == gene_duplicate.P_Value.min()]
    new_no_dup_df=no_dup_df.append(min_row)
new_no_dup_df

#reformat the table by getting rid of databaseIDs so we can join it to Endo and BRca
split = no_dup_df.index.str.split('_').str[0:2]
no_dup_df['new_index'] = split
# '_'.join(split)
no_dup_df['new_index'] = no_dup_df['new_index'].str.join('_')


brca_df = no_dup_df.set_index("new_index")


#are there duplicates in the index
brca_df


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


Name,Mutation,Location,Mutation_Status,ACAP2_S581_ESLPSTVsANSLYEPEGER_NP_036419.3,ACAP2_S581_SNDSGIQQSSDDGRESLPSTVsANSLYEPEGER_NP_036419.3,ACIN1_S657_SLsPGVSR_NP_055792.1|NP_001158286.1|NP_001158287.1,ACIN1_S657_SLsPGVSRDSSTSYTETK_NP_055792.1|NP_001158286.1,ACIN1_S825_IsVVSATK_NP_055792.1|NP_001158287.1|NP_001158288.1|NP_001158289.1,ACIN1_S825_IsVVSTK_NP_001158286.1,ACIN1_T682_DSSTSYTETKDPSSGQEVAtPPVPQLQVCEPK_NP_001158286.1|NP_001158287.1|NP_055792.1,...,ZC3H14_S343_TGSISSSVSVPAKPERRPsLPPSK_NP_079100.2|NP_001153575.1|NP_001313239.1|NP_001153576.1|NP_001313236.1|NP_001313225.1|NP_001313241.1|NP_001313227.1|NP_001313231.1|NP_997543.1|NP_997544.1|NP_001313234.1|NP_001313228.1|NP_001313226.1|NP_001313244.1|NP_001313230.1|NP_001313224.1|NP_001313229.1|NP_001313243.1|NP_001313232.1|NP_001313242.1|NP_001313235.1|NP_001313237.1|NP_001313233.1|NP_001313238.1|NP_001313240.1,ZC3H14_S343_TGSISSSVSVPAKPERRPsLPPSK_NP_079100.2|NP_001153575.1|NP_001313239.1|NP_001153576.1|NP_001313236.1|NP_001313225.1|NP_001313241.1|NP_001313229.1|NP_001313227.1|NP_001313231.1|NP_997543.1|NP_997544.1|NP_001313234.1|NP_001313228.1|NP_001313226.1|NP_001313244.1|NP_001313230.1|NP_001313224.1|NP_001313243.1|NP_001313232.1|NP_001313242.1|NP_001313235.1|NP_001313237.1|NP_001313233.1|NP_001313238.1|NP_001313240.1,ZC3H14_S665_RIPVLsPKPAVAPPAPPSSSQLCR_NP_079100.2|NP_001313239.1|NP_001313236.1|NP_001313241.1|NP_001313226.1|NP_001313230.1|NP_001313224.1|NP_001313229.1|NP_001313227.1|NP_997544.1|NP_001313234.1|NP_001313242.1|NP_001313237.1|NP_001313238.1,ZC3H14_S665_RIPVLsPKPVAPPAPPSSSQLCR_NP_001153575.1|NP_001153576.1|NP_001313225.1|NP_001313228.1|NP_001313244.1|NP_001313231.1|NP_997543.1|NP_001313243.1|NP_001313232.1|NP_001313235.1|NP_001313233.1|NP_001313240.1|NP_997545.2,ZCCHC8_S427_SSSHSSPGsPK_NP_060082.2,ZCCHC8_S427_SSSHSSPGsPKK_NP_060082.2,ZEB1_S323_TSQCSSPSLSASPGsPTRPQIR_NP_001167565.1|NP_001167564.1|NP_001167566.1|NP_001310603.1|NP_001310604.1|NP_001310607.1|NP_001310567.1|NP_001167567.1|NP_110378.3|NP_001310605.1|NP_001310606.1|NP_001121600.1,ZEB1_S323_TSQCSSPSLSASPGsPTRPQIR_NP_001167567.1|NP_110378.3|NP_001310605.1|NP_001310606.1|NP_001121600.1|NP_001167565.1|NP_001167564.1|NP_001167566.1|NP_001310603.1|NP_001310604.1|NP_001310607.1|NP_001310567.1,ZNF414_S296_SQGAGSsPR_NP_001139647.1|NP_115746.2,ZNF414_S296_SQGAGSsPRRPQGGSDAPSGACR_NP_115746.2
Patient_ID,,,,,,,,,,,,,,,,,,,,,
CPT000814,Amplification,Amplification,Single_mutation,0.4781,-0.7949,-4.1136,-3.4466,-1.6218,-2.7829,NaN,...,-1.7561,NaN,0.8268,1.1035,NaN,NaN,NaN,-1.8201,NaN,NaN
CPT001846,Missense_Mutation,p.E542K,Multiple_mutation,0.2503,-1.0130,-1.5451,-1.5511,-0.9719,-0.7861,NaN,...,-1.0021,NaN,1.4219,-0.0416,NaN,NaN,-0.2226,1.0117,-2.4053,NaN
X01BR001,Wildtype_Tumor,No_mutation,Wildtype_Tumor,NaN,NaN,-0.5980,-1.3631,0.2342,1.4307,0.5208,...,-1.0004,NaN,-0.6925,0.2617,-4.4618,-3.0199,0.1184,NaN,-1.8036,-3.5824
X01BR008,Wildtype_Tumor,No_mutation,Wildtype_Tumor,NaN,-0.0350,0.0348,0.6682,0.6015,1.0317,0.7745,...,0.3412,NaN,0.7698,0.4015,1.7777,NaN,NaN,0.3046,0.4221,NaN
X01BR009,Amplification,Amplification,Single_mutation,NaN,0.3861,0.6113,0.1563,-1.2738,-0.6842,0.3286,...,-0.9094,NaN,0.5100,0.2122,1.0981,NaN,NaN,0.3135,0.7217,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X21BR001,Amplification,Amplification,Single_mutation,NaN,0.1204,1.3387,0.6292,0.4579,0.3471,0.5478,...,0.4977,NaN,0.8767,-0.3365,0.2934,NaN,NaN,0.4302,-0.2482,NaN
X21BR002,Missense_Mutation,p.H1047R,Multiple_mutation,NaN,0.2408,0.6930,0.2026,0.6821,1.5682,0.6328,...,0.7277,NaN,-0.0363,0.1898,0.9027,NaN,NaN,0.8370,-0.1694,NaN
X21BR010,Missense_Mutation,p.H1047R,Multiple_mutation,1.1750,-0.1676,0.4896,1.1858,1.5798,2.4067,-0.3992,...,1.1934,0.4601,-0.1777,-0.7836,-2.7910,-1.6227,2.1904,2.1453,-0.9308,NaN


In [7]:
phosphoproteomics.columns.get_level_values('Name')

Index(['A2M', 'AAAS', 'AAAS', 'AAED1', 'AAGAB', 'AAGAB', 'AAK1', 'AAK1',
       'AAK1', 'AAK1',
       ...
       'ZZEF1', 'ZZZ3', 'ZZZ3', 'ZZZ3', 'ZZZ3', 'ZZZ3', 'ZZZ3', 'ZZZ3', 'ZZZ3',
       'ZZZ3'],
      dtype='object', name='Name', length=38775)

In [45]:
mut = br.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E545K', 'H1047R'])
phos = br.get_phosphoproteomics(tissue_type="tumor")


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


In [46]:
dropped = cptac.utils.reduce_multiindex(phos, ["Database_ID", "Peptide"])
dropped

dropped = br.reduce_multiindex(dropped, flatten=True)
dropped

fil = dropped.columns.get_level_values('Name').duplicated(False)
duplicates = phos[phos.columns[fil]]
no_duplicates = dropped[dropped.columns[~fil]]

len(duplicates.columns.get_level_values('Name').unique())
duplicates

cptac warning: Due to dropping the specified levels, dataframe now has 356 duplicated column headers. (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


Name                      ACAP2                                    \
Site                       S581                                     
Peptide     ESLPSTVsANSLYEPEGER SNDSGIQQSSDDGRESLPSTVsANSLYEPEGER   
Database_ID         NP_036419.3                       NP_036419.3   
CPT000814                0.4781                           -0.7949   
CPT001846                0.2503                           -1.0130   
X01BR001                    NaN                               NaN   
X01BR008                    NaN                           -0.0350   
X01BR009                    NaN                            0.3861   
...                         ...                               ...   
X21BR001                    NaN                            0.1204   
X21BR002                    NaN                            0.2408   
X21BR010                 1.1750                           -0.1676   
X22BR005                    NaN                           -0.5270   
X22BR006                 0.6632                               NaN   

Name                                            ACIN1  \
Site                                             S657   
Peptide                                      SLsPGVSR   
Database_ID NP_055792.1|NP_001158286.1|NP_001158287.1   
CPT000814                                     -4.1136   
CPT001846                                     -1.5451   
X01BR001                                      -0.5980   
X01BR008                                       0.0348   
X01BR009                                       0.6113   
...                                               ...   
X21BR001                                       1.3387   
X21BR002                                       0.6930   
X21BR010                                       0.4896   
X22BR005                                       0.6536   
X22BR006                                       0.1213   

Name                                    \
Site                                     
Peptide             SLsPGVSRDSSTSYTETK   
Database_ID NP_055792.1|NP_001158286.1   
CPT000814                      -3.4466   
CPT001846                      -1.5511   
X01BR001                       -1.3631   
X01BR008                        0.6682   
X01BR009                        0.1563   
...                                ...   
X21BR001                        0.6292   
X21BR002                        0.2026   
X21BR010                        1.1858   
X22BR005                       -1.0170   
X22BR006                        1.3795   

Name                                                                  \
Site                                                            S825   
Peptide                                                     IsVVSATK   
Database_ID NP_055792.1|NP_001158287.1|NP_001158288.1|NP_001158289.1   
CPT000814                                              -1.6218         
CPT001846                                              -0.9719         
X01BR001                                                0.2342         
X01BR008                                                0.6015         
X01BR009                                               -1.2738         
...                                                        ...         
X21BR001                                                0.4579         
X21BR002                                                0.6821         
X21BR010                                                1.5798         
X22BR005                                                0.1316         
X22BR006                                                0.6458         

Name                                                                  \
Site                                                            T682   
Peptide            IsVVSTK          DSSTSYTETKDPSSGQEVAtPPVPQLQVCEPK   
Database_ID NP_001158286.1 NP_001158286.1|NP_001158287.1|NP_055792.1   
CPT000814          -2.7829                                       NaN   
CPT001846          -0.7861          

In [9]:
#FOR NO DUPLICATES
#get the pval and min for no duplicates

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
mut_type = br.get_genotype_all_vars(gene)
# mut_type = mut_type.drop(columns = ["Mutation_Status", "Location"])

joined = mut_type.join(no_duplicates)
joined

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

braca_hotspot = br_hotspot_df.append(wildtype)
braca_hotspot = braca_hotspot.drop(columns = ["Mutation_Status", "Location"])

braca_hotspot


# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = braca_hotspot['Mutation'].isin(compare)
missense_wt = braca_hotspot[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


no_dup_d = {}

for prot in no_duplicates:
    dif_brca = wt_med[prot] - missense_med[prot]
    no_dup_d[prot] = dif_brca
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(no_dup_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
no_dup_df = median_diff.join(t_test)
no_dup_df

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:257: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


,Difference_In_Median,P_Value
A2M_S710,0.27610,0.747631
AAAS_S495,-0.22110,0.119525
AAAS_S541,0.35720,0.638884
AAED1_S12,-0.33260,0.159187
AAGAB_S310S311,0.44315,0.121689
...,...,...
ZZZ3_S397,-0.13435,0.761187
ZZZ3_S397T428N429,-0.39230,0.071512
ZZZ3_S606,0.18190,0.376978
ZZZ3_S82,0.01380,0.459838


# Colon

In [49]:
mut = cl.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E545K', 'H1047R'])
phos = cl.get_phosphoproteomics(tissue_type="tumor")
# phos  = cl.reduce_multiindex(phos, flatten=True)
phos

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


Name           AAAS                AAED1  AAGAB          AAK1                \
Site           S495   S525   S541    S12   S310   S311    S20    S21    S26   
Database_ID  Q9NRG9 Q9NRG9 Q9NRG9 Q7RTV5 Q6PD74 Q6PD74 Q2M2I8 Q2M2I8 Q2M2I8   
01CO005         NaN    NaN -0.240 -0.460    NaN    NaN -0.231    NaN    NaN   
01CO006     -0.3650    NaN    NaN -0.424 -0.015 -0.015    NaN -0.485    NaN   
01CO008      0.7250  0.137  0.137    NaN    NaN    NaN    NaN    NaN    NaN   
01CO013      0.2265    NaN    NaN -1.278  0.403  0.075 -0.223 -0.701    NaN   
01CO014      0.5600    NaN    NaN -0.382    NaN    NaN    NaN -0.259    NaN   
...             ...    ...    ...    ...    ...    ...    ...    ...    ...   
21CO006      0.0550    NaN    NaN  0.515    NaN    NaN    NaN    NaN    NaN   
21CO007         NaN    NaN    NaN  0.099    NaN    NaN    NaN -0.567    NaN   
22CO004      0.6220    NaN    NaN -0.308    NaN    NaN    NaN -0.137    NaN   
22CO006      0.6460  0.007  0.007    NaN    NaN    NaN    NaN    NaN    NaN   
27CO004      0.2130    NaN    NaN -0.478  0.642  0.340  0.300  0.003    NaN   

Name                ...  ZZEF1                                ZZZ3         \
Site          S618  ...  S1501   S1518  S1537  S1540  T1521   S113   S391   
Database_ID Q2M2I8  ... O43149  O43149 O43149 O43149 O43149 Q8IYH5 Q8IYH5   
01CO005        NaN  ...    NaN -0.6750 -1.404 -1.404    NaN -0.572    NaN   
01CO006        NaN  ...    NaN -0.2875  0.222  0.222 -0.701  0.624    NaN   
01CO008        NaN  ... -0.147 -0.1470    NaN    NaN    NaN    NaN    NaN   
01CO013        NaN  ... -0.041  0.0430  0.554  0.554  0.127  1.263    NaN   
01CO014        NaN  ...    NaN -0.1290 -0.919 -0.919    NaN  0.032    NaN   
...            ...  ...    ...     ...    ...    ...    ...    ...    ...   
21CO006        NaN  ...    NaN  0.1970 -0.339 -0.339  0.136    NaN    NaN   
21CO007        NaN  ...  0.070  0.0410 -0.077 -0.077    NaN  0.167    NaN   
22CO004        NaN  ...    NaN -0.2215  0.193  0.193 -0.347    NaN    NaN   
22CO006        NaN  ...  0.026  0.0260    NaN    NaN    NaN    NaN    NaN   
27CO004        NaN  ...  0.180  0.1100  0.108  0.108  0.040 -0.076    NaN   

Name                              
Site          S606    S90    S91  
Database_ID Q8IYH5 Q8IYH5 Q8IYH5  
01CO005      0.205    NaN    NaN  
01CO006        NaN    NaN    NaN  
01CO008        NaN    NaN -0.030  
01CO013        NaN    NaN    NaN  
01CO014      0.026    NaN    NaN  
...            ...    ...    ...  
21CO006        NaN    NaN    NaN  
21CO007      0.235  0.547    NaN  
22CO004      0.418    NaN    NaN  
22CO006        NaN    NaN -0.226  
27CO004        NaN    NaN    NaN  

[97 rows x 31339 columns]

In [60]:
dropped = cl.reduce_multiindex(phos, 'Database_ID')
dropped = cl.reduce_multiindex(dropped, flatten=True)

fil = dropped.columns.get_level_values('Name').duplicated(False)
duplicates = phos[phos.columns[fil]] #for duplicates we want to keep all levels, so grab from phos instead of dropped
no_duplicates = dropped[dropped.columns[~fil]]

len(duplicates.columns.get_level_values('Name').unique())
# cl.reduce_multiindex(duplicates, flatten=True)
duplicates

cptac warning: Due to dropping the specified levels, dataframe now has 150 duplicated column headers. (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


Name          ACIN1            ADD1          ARHGAP17          ARHGEF12  \
Site           S825            S600              S497               S41   
Database_ID  Q9UKV3 Q9UKV3-5 P35611 P35611-2   Q68EM7 Q68EM7-2   Q9NZN5   
01CO005     -1.5870   -0.880    NaN      NaN      NaN      NaN      NaN   
01CO006      0.4880    0.516    NaN      NaN   -0.597    0.311      NaN   
01CO008      0.1075   -0.549    NaN      NaN      NaN      NaN      NaN   
01CO013      0.7230    0.370    NaN      NaN      NaN      NaN      NaN   
01CO014     -1.5430   -0.444    NaN      NaN      NaN   -0.467      NaN   
...             ...      ...    ...      ...      ...      ...      ...   
21CO006      0.0870   -0.026    NaN      NaN      NaN      NaN      NaN   
21CO007     -0.0170      NaN    NaN      NaN      NaN      NaN      NaN   
22CO004      0.6310    0.523    NaN      NaN      NaN      NaN      NaN   
22CO006     -0.0065   -0.114    NaN      NaN      NaN      NaN      NaN   
27CO004      0.6050    0.318    NaN      NaN      NaN      NaN      NaN   

Name                 ARL6IP4           ... THEMIS            TJP1           \
Site                    S231           ...   S584            S912            
Database_ID Q9NZN5-2  Q66PJ3 Q66PJ3-2  ... Q8N1K5 Q8N1K5-4 Q07157 Q07157-2   
01CO005          NaN     NaN      NaN  ... -1.407      NaN -1.897   -1.878   
01CO006        0.048   0.324      NaN  ...    NaN    0.002 -1.079   -0.549   
01CO008       -0.527     NaN      NaN  ...    NaN      NaN  0.225    0.116   
01CO013       -0.174     NaN      NaN  ...  0.457   -0.618 -0.566   -0.665   
01CO014          NaN     NaN      NaN  ...    NaN      NaN -0.181   -0.899   
...              ...     ...      ...  ...    ...      ...    ...      ...   
21CO006        0.195     NaN      NaN  ...    NaN      NaN -0.259    0.326   
21CO007          NaN  -0.625    0.216  ...    NaN      NaN  0.203   -0.510   
22CO004        0.264     NaN   -0.297  ...    NaN      NaN  0.399   -0.839   
22CO006        0.321     NaN      NaN  ...    NaN      NaN -0.596    0.820   
27CO004        0.124     NaN      NaN  ...  0.038   -0.838 -0.454    0.749   

Name          TMPO         TPD52L2          ZC3H14           
Site          S184             S96            S665           
Database_ID P42166  P42167  O43399 O43399-2 Q6PJT7 Q6PJT7-2  
01CO005     -0.412 -0.1240     NaN      NaN -1.258      NaN  
01CO006     -1.244 -0.0870  -0.748      NaN    NaN      NaN  
01CO008      0.466  0.8870     NaN      NaN    NaN      NaN  
01CO013     -1.290 -0.7400  -0.627      NaN    NaN      NaN  
01CO014     -0.119 -0.4815     NaN      NaN    NaN      NaN  
...            ...     ...     ...      ...    ...      ...  
21CO006      0.135 -0.4340     NaN      NaN    NaN      NaN  
21CO007      0.188  0.0855     NaN      NaN  0.486      NaN  
22CO004      0.512 -0.0350     NaN      NaN    NaN      NaN  
22CO006     -0.044 -0.0200     NaN      NaN    NaN      NaN  
27CO004     -1.251 -0.5230  -0.716      NaN    NaN      NaN  

[97 rows x 150 columns]

In [59]:
#FOR NO DUPLICATES
#get the pval and min for no duplicates

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
mut_type = cl.get_genotype_all_vars(gene)
# mut_type = mut_type.drop(columns = ["Mutation_Status", "Location"])

joined = mut_type.join(no_duplicates)
joined

#select samples containing hotspot mutations
cl_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

colon_hotspot = cl_hotspot_df.append(wildtype)
colon_hotspot = colon_hotspot.drop(columns = ["Mutation_Status", "Location"])

colon_hotspot

# Keep two values to compare
compare = ['Wildtype_Tumor','nonsynonymous SNV']
get = colon_hotspot['Mutation'].isin(compare)
missense_wt = colon_hotspot[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "nonsynonymous SNV"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


no_dup_d = {}

for prot in no_duplicates:
    dif_colon = wt_med[prot] - missense_med[prot]
    no_dup_d[prot] = dif_colon
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(no_dup_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
no_dup_df = median_diff.join(t_test)
no_dup_df

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  rejec

,Difference_In_Median,P_Value
AAAS_S495,0.12500,0.356935
AAAS_S525,0.53400,NaN
AAAS_S541,0.41475,0.159858
AAED1_S12,-0.07200,0.945226
AAGAB_S310,NaN,NaN
...,...,...
ZZZ3_S113,-0.14900,0.733776
ZZZ3_S391,-0.04600,NaN
ZZZ3_S606,-0.11600,0.458431
ZZZ3_S90,NaN,NaN


In [62]:
#DEALING WITH DUPLICATES
#get the pval and min for duplicates

duplicates = cl.reduce_multiindex(duplicates, flatten=True)

# We need to figure which of databaseID we want to use, for each protein


#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest

mut_type = cl.get_genotype_all_vars(gene)
joined = mut_type.join(duplicates)
joined

#select samples containing hotspot mutations
cl_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

colon_hotspot = cl_hotspot_df.append(wildtype)
colon_hotspot = colon_hotspot.drop(columns = ["Mutation_Status", "Location"])
colon_hotspot

# Keep two values to compare
compare = ['Wildtype_Tumor','nonsynonymous SNV']
get = colon_hotspot['Mutation'].isin(compare)
missense_wt = colon_hotspot[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "nonsynonymous SNV"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype

duplicates

duplicates_d = {}

for prot in duplicates:
    dif_brca = wt_med[prot] - missense_med[prot]
    duplicates_d[prot] = dif_brca
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(duplicates_d, orient='index', columns=['Difference_In_Median'])
median_diff

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
dup_df = median_diff.join(t_test)


duplicate_gene_names = list(set(duplicates.columns.get_level_values('Name'))) #get a list of genes names that are duplicated
# duplicate_gene_names

for name in duplicate_gene_names:
    gene_duplicate = dup_df[dup_df.index.str.contains(name)]
    min_row = gene_duplicate[gene_duplicate.P_Value == gene_duplicate.P_Value.min()]
    new_no_dup_df=no_dup_df.append(min_row)
new_no_dup_df

#reformat the table by getting rid of databaseIDs so we can join it to Endo and BRca
split = no_dup_df.index.str.split('_').str[0:2]
no_dup_df['new_index'] = split
# '_'.join(split)
no_dup_df['new_index'] = no_dup_df['new_index'].str.join('_')


colon_df = no_dup_df.set_index("new_index")


#are there duplicates in the index
colon_df



cptac warning: You tried to flatten an index that didn't have multiple levels, so we didn't actually change anything. (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 4)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in d

,Difference_In_Median,P_Value
new_index,,
AAAS_S495,0.12500,0.356935
AAAS_S525,0.53400,NaN
AAAS_S541,0.41475,0.159858
AAED1_S12,-0.07200,0.945226
AAGAB_S310,NaN,NaN
...,...,...
ZZZ3_S113,-0.14900,0.733776
ZZZ3_S391,-0.04600,NaN
ZZZ3_S606,-0.11600,0.458431


# Join Endo, Brca, and Colon together


In [63]:
brca_df=brca_df.rename_axis(None)

In [64]:
brca_endo = brca_df.join(endo_df, lsuffix='_Brca', rsuffix='_Endo')
brca_endo

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo
A2M_S710,0.27610,0.747631,NaN,NaN
AAAS_S495,-0.22110,0.119525,0.257295,0.650738
AAAS_S541,0.35720,0.638884,-0.061000,NaN
AAED1_S12,-0.33260,0.159187,0.378000,0.168105
AAGAB_S310S311,0.44315,0.121689,NaN,NaN
...,...,...,...,...
ZZZ3_S397,-0.13435,0.761187,0.062100,0.529028
ZZZ3_S397T428N429,-0.39230,0.071512,NaN,NaN
ZZZ3_S606,0.18190,0.376978,NaN,NaN
ZZZ3_S82,0.01380,0.459838,NaN,NaN


In [65]:
final_table = brca_endo.join(colon_df, rsuffix='_colon')
final_table

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
A2M_S710,0.27610,0.747631,NaN,NaN,NaN,NaN
AAAS_S495,-0.22110,0.119525,0.257295,0.650738,0.12500,0.356935
AAAS_S541,0.35720,0.638884,-0.061000,NaN,0.41475,0.159858
AAED1_S12,-0.33260,0.159187,0.378000,0.168105,-0.07200,0.945226
AAGAB_S310S311,0.44315,0.121689,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
ZZZ3_S397,-0.13435,0.761187,0.062100,0.529028,NaN,NaN
ZZZ3_S397T428N429,-0.39230,0.071512,NaN,NaN,NaN,NaN
ZZZ3_S606,0.18190,0.376978,NaN,NaN,-0.11600,0.458431
ZZZ3_S82,0.01380,0.459838,NaN,NaN,NaN,NaN


In [66]:
#only keep significant pvals
significant = final_table.loc[(final_table['P_Value_Brca'] <= .05) |
                (final_table['P_Value_Endo'] <= .05) |
                (final_table['P_Value'] <= .05)]
significant

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
AAGAB_S311,0.30040,0.704996,0.457000,0.569183,-0.6655,0.014602
AAK1_T606,0.24840,0.512914,0.570100,0.002949,-0.0205,0.814122
AAK1_T674,-0.44810,0.480089,0.573000,0.002438,-0.2455,0.950929
AATF_S153,0.69170,0.025235,-0.085200,0.177239,0.2540,0.230845
AATF_S316S320S321,0.30915,0.038469,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
ZRANB2_S83,0.00595,0.797180,-0.198100,0.033825,NaN,NaN
ZRANB2_T90,0.16985,0.643431,-3.050000,0.029035,NaN,NaN
ZRSR2_S384,-0.19190,0.975425,-0.213977,0.006471,0.2300,0.676926
ZYX_S290,-0.14015,0.473074,0.295000,0.032401,NaN,NaN


In [68]:
significant.to_csv("phosphoproteomics_trans.csv")